# ResNet Classification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Charl-AI/Solstice/blob/main/docs/from_flax_to_solstice.ipynb)


The ResNet example demonstrates how to perform image classification on CIFAR10 with
data-parallel multi-gpu support. Uses Haiku to define the base neural net and
[Hydra](https://hydra.cc/docs/intro/) for config management.

!!! faq
    Run `python examples/resnet_classification.py -h` to see the available configuration
    options.

    Logging is done with TensorBoard. Run `tensorboard --logdir=outputs` to view the
    logs.

!!! summary
    This example demonstrates:

    - How to implement `solstice.Experiment` for training a ResNet50 with multi-gpu
        support.

    - Usage of `solstice.ClassificationMetrics` for tracking metrics.

    - Usage of `solstice.LoggingCallback` with TensorBoard (using the
        [CLU](https://github.com/google/CommonLoopUtils) SummaryWriter interface).

    - Usage of `solstice.ProfilingCallback` for profiling with TensorBoard.

!!! warning
    Multi-GPU support is not yet implemented. Currently only working with single GPU.

In [1]:
%pip install solstice-jax
%pip install dm-haiku
%pip install optax
%pip install clu


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import logging
import tensorflow as tf

# Ensure TF does not see GPU and grab all GPU memory.
tf.config.set_visible_devices([], device_type="GPU")
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"  # JAX memory preallocation
logging.getLogger("absl").setLevel(logging.WARNING)  # Keep TF/JAX quiet

2022-07-18 19:42:38.857924: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-18 19:42:38.861506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-18 19:42:38.861517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-18 19:42:40.291715: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-18 19:42:40.291741: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

## Step 1: create the Solstice Experiment

In [3]:
from typing import Tuple
import solstice
import equinox as eqx
import optax
import jax
import jax.numpy as jnp
import haiku as hk

class ResNetClassifier(solstice.Experiment):
    """A ResNet50 image classifier for multiclass problems. You can use it with binary
    problems by treating it as a multiclass case with 2 classes and a threshold fixed
    at 0.5."""

    params: optax.Params
    model_state: optax.OptState
    opt_state: optax.OptState
    model_fns: hk.TransformedWithState
    opt: optax.GradientTransformation
    num_classes: int

    def __init__(
        self,
        optimizer: optax.GradientTransformation,
        num_classes: int,
        input_shape: Tuple[int, int, int],
        *,
        rng: int,
    ):
        assert num_classes > 1, f"Only multiclass (>=2) supported, got {num_classes}"
        key = jax.random.PRNGKey(rng)
        dummy_batch = jnp.zeros((1, *input_shape))
        self.model_fns = hk.without_apply_rng(
            hk.transform_with_state(
                lambda x, is_training: hk.nets.ResNet50(num_classes, resnet_v2=True)(
                    x, is_training
                ),
            )
        )
        self.params, self.model_state = self.model_fns.init(
            key, dummy_batch, is_training=True
        )

        self.opt = optimizer
        self.opt_state = self.opt.init(self.params)
        self.num_classes = num_classes

    @eqx.filter_jit(kwargs=dict(batch=True))
    def train_step(
        self, batch: Tuple[jnp.ndarray, jnp.ndarray]
    ) -> Tuple[solstice.Experiment, solstice.Metrics]:
        imgs, labels = batch

        def loss_fn(params):
            logits, new_model_state = self.model_fns.apply(
                params, self.model_state, imgs, is_training=True
            )
            loss = jnp.mean(
                optax.softmax_cross_entropy(
                    logits, jax.nn.one_hot(labels, self.num_classes)
                )
            )
            return loss, (logits, new_model_state)

        (loss, (logits, new_model_state)), grads = jax.value_and_grad(
            loss_fn, has_aux=True
        )(self.params)

        updates, new_opt_state = self.opt.update(grads, self.opt_state, self.params)
        new_params = optax.apply_updates(self.params, updates)

        preds = jnp.argmax(logits, axis=-1)
        metrics = solstice.ClassificationMetrics(preds, labels, loss, self.num_classes)

        return (
            solstice.replace(
                self,
                params=new_params,
                opt_state=new_opt_state,
                model_state=new_model_state,
            ),
            metrics,
        )

    @eqx.filter_jit(kwargs=dict(batch=True))
    def eval_step(
        self, batch: Tuple[jnp.ndarray, jnp.ndarray]
    ) -> Tuple[solstice.Experiment, solstice.Metrics]:
        imgs, labels = batch

        logits, _ = self.model_fns.apply(
            self.params, self.model_state, imgs, is_training=False
        )
        loss = jnp.mean(
            optax.softmax_cross_entropy(
                logits, jax.nn.one_hot(labels, self.num_classes)
            )
        )
        preds = jnp.argmax(logits, axis=-1)
        metrics = solstice.ClassificationMetrics(preds, labels, loss, self.num_classes)

        return self, metrics


## Step 2: set up a callback for TensorBoard logging

In [4]:
from typing import Mapping
from typing_extensions import Literal
from clu import metric_writers

def create_tensorboard_callback(log_dir: str, log_every_n_steps:int
) -> solstice.Callback:
    
    logger = metric_writers.create_default_writer(log_dir)
    # this gets passed to `solstice.LoggingCallback` and tells it what to do with the
    # output of `metrics.compute()`
    def logging_fn(
        metrics_dict: Mapping[str, float],
        step: int,
        mode: Literal["train", "val", "test"],
    ):
        logger.write_scalars(
            step,
            {f"{mode}/{key}": val for key, val in metrics_dict.items()},
        )

    return solstice.LoggingCallback(log_every_n_steps, logging_fn)

## Step 3: set up the dataset and config (not that relevant to Solstice)

In [5]:
import dataclasses
import functools
import tensorflow_datasets as tfds

def get_cifar_datasets(
    data_dir: str, batch_size: int, prefetch_size: int
) -> Tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    load_ds = functools.partial(
        tfds.load, data_dir=data_dir, as_supervised=True, name="cifar10"
    )

    train_ds = load_ds(split="train[:80%]")
    val_ds = load_ds(split="train[80%:]")
    test_ds = load_ds(split="test")
    assert isinstance(train_ds, tf.data.Dataset)  # just for type narrowing
    assert isinstance(val_ds, tf.data.Dataset)
    assert isinstance(test_ds, tf.data.Dataset)

    preprocess_data = lambda x, y: (
        tf.cast(x, tf.float32) / 255.0,
        tf.cast(y, tf.float32),
    )
    prepare_data = (
        lambda ds: ds.map(
            preprocess_data, num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        .batch(batch_size)
        .prefetch(prefetch_size)
        .cache()
    )
    train_ds = prepare_data(
        train_ds.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
    )

    val_ds = prepare_data(val_ds)
    test_ds = prepare_data(test_ds)

    return train_ds, val_ds, test_ds

@dataclasses.dataclass
class Config:
    rng: int = 0
    num_epochs: int = 50
    lr: float = 4e-2

    data_dir: str = "/tmp/data"
    batch_size: int = 128
    prefetch_size: int = 5

    log_dir: str = "logs/"
    log_every_n_steps: int = 50


/opt/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 4: define a main function to run everything

In [6]:
def main(cfg: Config):
    train_ds, val_ds, test_ds = get_cifar_datasets(
        cfg.data_dir, cfg.batch_size, cfg.prefetch_size
    )
    optimizer: optax.GradientTransformation = optax.adamw(learning_rate=cfg.lr)

    exp = ResNetClassifier(
        optimizer=optimizer,
        num_classes=10,  # hard coded to cifar10 classes and shape
        input_shape=(32, 32, 3),
        rng=cfg.rng,
    )

    logger = metric_writers.create_default_writer(cfg.log_dir)
    # this gets passed to `solstice.LoggingCallback` and tells it what to do with the
    # output of `metrics.compute()`
    def logging_fn(
        metrics_dict: Mapping[str, float],
        step: int,
        mode: Literal["train", "val", "test"],
    ):
        logger.write_scalars(
            step,
            {f"{mode}/{key}": val for key, val in metrics_dict.items()},
        )

    exp = solstice.train(
        exp,
        num_epochs=cfg.num_epochs,
        train_ds=train_ds,
        val_ds=val_ds,
        callbacks=[
            solstice.ProfilingCallback(cfg.log_dir, steps_to_profile=[1, 2, 3]),
            solstice.LoggingCallback(
                log_every_n_steps=cfg.log_every_n_steps, logging_fn=logging_fn
            ),
        ],
    )

    _ = solstice.test(
        exp,
        test_ds=test_ds,
        callbacks=[solstice.LoggingCallback(logging_fn=logging_fn)],
    )

## Step 5: run the experiment and visualise the results

In [7]:
main(Config())

%tensorboard --logdir logs/

2022-07-18 19:42:45.526989: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:07<00:00,  7.10s/ url]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]2022-07-18 19:42:55.973888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dataset cifar10 downloaded and prepared to /tmp/data/cifar10/3.0.2. Subsequent calls will reuse this data.


KeyboardInterrupt: 